# **Homework**: Data talks club data engineering zoomcamp Data loading workshop

Hello folks, let's practice what we learned - Loading data with the best practices of data engineering.

Here are the exercises we will do




# 1. Use a generator

Remember the concept of generator? Let's practice using them to futher our understanding of how they work.

Let's define a generator and then run it as practice.

**Answer the following questions:**

- **Question 1: What is the sum of the outputs of the generator for limit = 5?**
- **Question 2: What is the 13th number yielded**

I suggest practicing these questions without GPT as the purpose is to further your learning.

In [3]:
def square_root_generator(limit):
    n = 1
    while n <= limit:
        yield n ** 0.5
        n += 1

# Example usage:
limit = 5
generator = square_root_generator(limit)

##modyfing the code to get the answer for q1

number = []
for sqrt_value in generator:
    number.append((sqrt_value))

print(sum(number))


8.382332347441762


In [4]:
#question 2

def square_root_generator(limit):
    n = 1
    while n <= limit:
        yield n ** 0.5
        n += 1

# Example usage:
limit = 13
generator = square_root_generator(limit)

##modyfing the code to get the answer for q1

number = []
for sqrt_value in generator:
    number.append((sqrt_value))

print(number[12])



3.605551275463989


# 2. Append a generator to a table with existing data


Below you have 2 generators. You will be tasked to load them to duckdb and answer some questions from the data

1. Load the first generator and calculate the sum of ages of all people. Make sure to only load it once.
2. Append the second generator to the same table as the first.
3. **After correctly appending the data, calculate the sum of all ages of people.**




In [5]:
def people_1():
    for i in range(1, 6): 
        yield {"ID": i, "Name": f"Person_{i}", "Age": 25 + i, "City": "City_A"}

for person in people_1():
         print(person)

def people_2():
    for i in range(3, 9):
         yield {"ID": i, "Name": f"Person_{i}", "Age": 30 + i, "City": "City_B", "Occupation": f"Job_{i}"}

for person in people_2():
     print(person)

{'ID': 1, 'Name': 'Person_1', 'Age': 26, 'City': 'City_A'}
{'ID': 2, 'Name': 'Person_2', 'Age': 27, 'City': 'City_A'}
{'ID': 3, 'Name': 'Person_3', 'Age': 28, 'City': 'City_A'}
{'ID': 4, 'Name': 'Person_4', 'Age': 29, 'City': 'City_A'}
{'ID': 5, 'Name': 'Person_5', 'Age': 30, 'City': 'City_A'}
{'ID': 3, 'Name': 'Person_3', 'Age': 33, 'City': 'City_B', 'Occupation': 'Job_3'}
{'ID': 4, 'Name': 'Person_4', 'Age': 34, 'City': 'City_B', 'Occupation': 'Job_4'}
{'ID': 5, 'Name': 'Person_5', 'Age': 35, 'City': 'City_B', 'Occupation': 'Job_5'}
{'ID': 6, 'Name': 'Person_6', 'Age': 36, 'City': 'City_B', 'Occupation': 'Job_6'}
{'ID': 7, 'Name': 'Person_7', 'Age': 37, 'City': 'City_B', 'Occupation': 'Job_7'}
{'ID': 8, 'Name': 'Person_8', 'Age': 38, 'City': 'City_B', 'Occupation': 'Job_8'}



1. Load the first generator and calculate the sum of ages of all people. Make sure to only load it once.

In [6]:
import dlt
import duckdb

def people_1():
    for i in range(1, 6): 
        yield {"ID": i, "Name": f"Person_{i}", "Age": 25 + i, "City": "City_A"}

for person in people_1():
         print(person)


# define the connection to load to.
# We now use duckdb, but you can switch to Bigquery later
people1_pipeline = dlt.pipeline(destination='duckdb', dataset_name='people1_pipeline')


#people1_pipeline
# run with merge write disposition.
# This is so scaffolding is created for the next example,
# where we look at merging data

#load data pipeline
people1_pipeline.run(people_1(),
                 table_name='table_people_1',
                 write_disposition="replace")

{'ID': 1, 'Name': 'Person_1', 'Age': 26, 'City': 'City_A'}
{'ID': 2, 'Name': 'Person_2', 'Age': 27, 'City': 'City_A'}
{'ID': 3, 'Name': 'Person_3', 'Age': 28, 'City': 'City_A'}
{'ID': 4, 'Name': 'Person_4', 'Age': 29, 'City': 'City_A'}
{'ID': 5, 'Name': 'Person_5', 'Age': 30, 'City': 'City_A'}


LoadInfo(pipeline=<dlt.pipeline.pipeline.Pipeline object at 0x152fa3d50>, metrics={'1708011485.851731': [{'started_at': DateTime(2024, 2, 15, 15, 38, 6, 186965, tzinfo=Timezone('UTC')), 'finished_at': DateTime(2024, 2, 15, 15, 38, 6, 349070, tzinfo=Timezone('UTC'))}]}, destination_type='dlt.destinations.duckdb', destination_displayable_credentials='duckdb:////Users/juancarlosgarcia/Desktop/DE Zoomcamp 2024/Homework/Workshop - Data Ingestion/dlt_resources/dlt_ipykernel_launcher.duckdb', destination_name='duckdb', environment=None, staging_type=None, staging_name=None, staging_displayable_credentials=None, destination_fingerprint='', dataset_name='people1_pipeline', loads_ids=['1708011485.851731'], load_packages=[LoadPackageInfo(load_id='1708011485.851731', package_path='/Users/juancarlosgarcia/.dlt/pipelines/dlt_ipykernel_launcher/load/loaded/1708011485.851731', state='loaded', schema=Schema dlt_ipykernel_launcher at 5691116368, schema_update={}, completed_at=DateTime(2024, 2, 15, 15, 3

In [7]:
conn = duckdb.connect(f"{people1_pipeline.pipeline_name}.duckdb")

# loading table
conn.sql(f"SET search_path = '{people1_pipeline.dataset_name}'")
print('Loaded tables: ')
display(conn.sql("show tables"))

## load the first generator and calculate the sum of all people ages
print("\n\n\n Sum of all people ages")
sum_ages_q1 = conn.sql("SELECT sum(age) As ages_sum FROM table_people_1").df()
display(sum_ages_q1)

Loaded tables: 


┌─────────────────────┐
│        name         │
│       varchar       │
├─────────────────────┤
│ _dlt_loads          │
│ _dlt_pipeline_state │
│ _dlt_version        │
│ table_people1       │
│ table_people_1      │
└─────────────────────┘




 Sum of all people ages


,ages_sum
0,140.0


2 - Append the second generator to the same table as the first.

In [8]:
#load data pipeline
people1_pipeline.run(people_2(),
                 table_name='table_people_1',
                 write_disposition="append")

LoadInfo(pipeline=<dlt.pipeline.pipeline.Pipeline object at 0x152fa3d50>, metrics={'1708011534.524779': [{'started_at': DateTime(2024, 2, 15, 15, 38, 54, 880811, tzinfo=Timezone('UTC')), 'finished_at': DateTime(2024, 2, 15, 15, 38, 55, 63934, tzinfo=Timezone('UTC'))}]}, destination_type='dlt.destinations.duckdb', destination_displayable_credentials='duckdb:////Users/juancarlosgarcia/Desktop/DE Zoomcamp 2024/Homework/Workshop - Data Ingestion/dlt_resources/dlt_ipykernel_launcher.duckdb', destination_name='duckdb', environment=None, staging_type=None, staging_name=None, staging_displayable_credentials=None, destination_fingerprint='', dataset_name='people1_pipeline', loads_ids=['1708011534.524779'], load_packages=[LoadPackageInfo(load_id='1708011534.524779', package_path='/Users/juancarlosgarcia/.dlt/pipelines/dlt_ipykernel_launcher/load/loaded/1708011534.524779', state='loaded', schema=Schema dlt_ipykernel_launcher at 5680324752, schema_update={}, completed_at=DateTime(2024, 2, 15, 15, 

3- After correctly appending the data, calculate the sum of all ages of people.

In [9]:
#Question 3s
sum_ages_q3= conn.sql("SELECT sum(age) AS sum_age FROM table_people_1")
display(sum_ages_q3)


┌─────────┐
│ sum_age │
│ int128  │
├─────────┤
│     353 │
└─────────┘

# 3. Merge a generator

Re-use the generators from Exercise 2.

A table's primary key needs to be created from the start, so load your data to a new table with primary key ID.

Load your first generator first, and then load the second one with merge. Since they have overlapping IDs, some of the records from the first load should be replaced by the ones from the second load.

After loading, you should have a total of 8 records, and ID 3 should have age 33.

Question: **Calculate the sum of ages of all the people loaded as described above.**


# Solution: First make sure that the following modules are installed:

In [ ]:
#Install the dependencies
%%capture
!pip install dlt[duckdb]

In [10]:
#load table 1
people1_pipeline.run(people_1(),
                    table_name='table_people_1',
                    write_disposition="replace")

#load table 2 using merge and primery key [id]
people1_pipeline.run(people_2(),
                    table_name='table_people_1',
                    write_disposition="merge",
                    primary_key="id")
sum_ages_q4 = conn.sql("SELECT sum(age) AS sum_age FROM table_people_1")
display(sum_ages_q4)

┌─────────┐
│ sum_age │
│ int128  │
├─────────┤
│     266 │
└─────────┘

Questions? difficulties? We are here to help.
- DTC data engineering course channel: https://datatalks-club.slack.com/archives/C01FABYF2RG
- dlt's DTC cohort channel: https://dlthub-community.slack.com/archives/C06GAEX2VNX